# Data Wrangling

Due to the dimensions of this dataset, it was considered better to have a separate notebook for data Wrangling. At this stage I tried to get a dataset clean and tiddy enough for the subsequent analysis work.

In [1]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

## Data Gathering

Open `pisa2012.csv`

In [2]:
pisa_df_ = pd.read_csv('pisa2012.csv', encoding='windows-1252') 

/Users/home/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (15,16,17,21,22,23,24,25,26,30,31,36,37,45,65,123,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,475) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


There are problems to read certain columns. I will try to get the type of these columns and reopen the csv specifying the type of each column.

In [4]:
col_dtype_series = pisa_df_.dtypes

Now that we have a series with the types of each column, I will save it as a `pkl` file so that the next time I open this notebook, I'll just import the type of each columns and specify it when reading `pisa2012.csv`.

In [5]:
col_dtype_series.to_pickle('col_dtype.pkl')

Read `col_dtype.pkl` and convert it to a dictionary in order to open `pisa2012.csv` successfully.

In [6]:
col_dtype_series = pd.read_pickle("col_dtype.pkl")
col_dtype_dict = col_dtype_series.to_dict()

# Read pisa2012.csv and pisadict2012.csv
pisa_df = pd.read_csv('pisa2012.csv', encoding='windows-1252', dtype= col_dtype_dict)
pisa_csv = pd.read_csv('pisadict2012.csv', encoding='windows-1252')

# Convert pisa_csv to a dictionary
pisa_dict = pd.Series(pisa_csv.x.values, index = pisa_csv['Unnamed: 0']).to_dict()

In [7]:
# Get a general view of this df
print(pisa_df.shape)
pisa_df.sample(3)

(485490, 636)


,Unnamed: 0,CNT,SUBNATIO,STRATUM,OECD,NC,SCHOOLID,STIDSTD,ST01Q01,ST02Q01,...,W_FSTR75,W_FSTR76,W_FSTR77,W_FSTR78,W_FSTR79,W_FSTR80,WVARSTRR,VAR_UNIT,SENWGT_STU,VER_STU
385011,385012,Portugal,6200000,PRT0006,OECD,Portugal,52,1483,7,1.0,...,7.3668,2.4556,7.3668,7.3668,2.4556,2.4556,60,2,0.0511,22NOV13
448343,448344,Chinese Taipei,1580000,TAP9797,Non-OECD,Chinese Taipei,1,13,10,2.0,...,29.7750,29.7750,29.7750,29.7750,29.7750,29.7750,58,1,0.2054,22NOV13
177286,177287,Finland,2460000,FIN0017,OECD,Finland,58,1612,9,1.0,...,1.4238,1.4373,1.4238,4.5938,4.5359,1.5120,17,2,0.0491,22NOV13


One of the most striking things about this df is the number of columns it has. Working with so many columns would be an error, since many of these columns will not be useful to answer the questions that we want to answer or look for the insights we want. That's why, it will be best to remove those columns. In order to do this, first we have to give a view to each column and formulate the points that interest us, in order to define which columns will not be revelant to our work.

...

After seeing each column to know what data they provide, I have written down the following points that I find interesting:

- What are the differences between the results of boys and girls?
- Are there statistically significant differences between foreign students and local students?
- Is there any relationship between students who arrive late, skip a class or miss a whole school day and a lower performance on the study?
- Is there any difference in results between students who have access to the internet and those who do not?
- Is there any relationship between the economic-socio-cultural level of the student and his performance in the test?

Keeping this in mind, and after having read column by column, the columns that are of interest and useful to resolve these points are the following:

In [8]:
all_cols = ['STIDSTD', 'ST04Q01', 'CNT', 'ESCS', 'PV1SCIE', 'PV1READ',  'PV1MATH', 
             'IMMIG', 'ST08Q01', 'ST09Q01', 'ST115Q01', 'IC01Q04', 'ST26Q06']

# Print each column
for col_to_keep in all_cols:
    print(col_to_keep, pisa_dict[col_to_keep])

STIDSTD Student ID
ST04Q01 Gender
CNT Country code 3-character
ESCS Index of economic, social and cultural status
PV1SCIE Plausible value 1 in science
PV1READ Plausible value 1 in reading
PV1MATH Plausible value 1 in mathematics
IMMIG Immigration status
ST08Q01 Truancy - Late for School
ST09Q01 Truancy - Skip whole school day
ST115Q01 Truancy - Skip classes within school day
IC01Q04 At Home - Internet connection
ST26Q06 Possessions - Internet


## Data Assessing

In [9]:
# Get df with needed columns
all_cols_df = pisa_df[all_cols]

In [10]:
all_cols_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485490 entries, 0 to 485489
Data columns (total 13 columns):
STIDSTD     485490 non-null int64
ST04Q01     485490 non-null object
CNT         485490 non-null object
ESCS        473648 non-null float64
PV1SCIE     485490 non-null float64
PV1READ     485490 non-null float64
PV1MATH     485490 non-null float64
IMMIG       471793 non-null object
ST08Q01     479143 non-null object
ST09Q01     479131 non-null object
ST115Q01    479269 non-null float64
IC01Q04     297305 non-null object
ST26Q06     473182 non-null object
dtypes: float64(5), int64(1), object(7)
memory usage: 48.2+ MB


In [11]:
# Get unique values for each column
for col in all_cols_df.columns:
    print ("________" * 10, "\n", col, pisa_dict[col], all_cols_df[col].unique())

________________________________________________________________________________ 
 STIDSTD Student ID [    1     2     3 ... 33804 33805 33806]
________________________________________________________________________________ 
 ST04Q01 Gender ['Female' 'Male']
________________________________________________________________________________ 
 CNT Country code 3-character ['Albania' 'United Arab Emirates' 'Argentina' 'Australia' 'Austria'
 'Belgium' 'Bulgaria' 'Brazil' 'Canada' 'Switzerland' 'Chile' 'Colombia'
 'Costa Rica' 'Czech Republic' 'Germany' 'Denmark' 'Spain' 'Estonia'
 'Finland' 'France' 'United Kingdom' 'Greece' 'Hong Kong-China' 'Croatia'
 'Hungary' 'Indonesia' 'Ireland' 'Iceland' 'Israel' 'Italy' 'Jordan'
 'Japan' 'Kazakhstan' 'Korea' 'Liechtenstein' 'Lithuania' 'Luxembourg'
 'Latvia' 'Macao-China' 'Mexico' 'Montenegro' 'Malaysia' 'Netherlands'
 'Norway' 'New Zealand' 'Peru' 'Poland' 'Portugal' 'Qatar'
 'China-Shanghai' 'Perm(Russian Federation)' 'Florida (USA)'
 'Connecticut

* `STIDSTD` (student id) should be a string. We do not want to do mathematical operations with the id of the students.
* In `CNT`, 'Perm(Russian Federation)' should be 'Russian Federation'; 'Macao-China', 'China-Shanghai', 'Hong Kong-China' -> 'China'; 'Florida (USA)', 'Connecticut (USA)', 'Massachusetts (USA)' -> 'United States of America'; Chinese Taipei' -> Taiwan.
* We could change the numerical values of `ST115Q01` by the definition of those numbers.
* `ST04Q01` is a categorical variable.
* `IMMIG`, `IC01Q04`, `ST08Q01`, `ST09Q01` are categorical ordinal variables, not strings.
* Values of `ST08Q01` and `ST09Q01` have more spaces than necessary. Example: 'None&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;'.
* `ST26Q06` 'Possessions - Internet' and `IC01Q04` 'At Home - Internet connection' seem to represent the same.

There are several columns that have null values. I decided that I will leave those columns as they are and I will take care of those null values depending on the case.

## Clean

**`STIDSTD` (student id) should be a string. We do not want to do mathematical operations with the id of the students.**

**Define:** Change to `str` using `astype(str)`

**Code:**

In [12]:
all_cols_df.STIDSTD = all_cols_df.STIDSTD.astype(str)

/Users/home/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


**Check:**

In [13]:
all_cols_df.STIDSTD.dtypes

dtype('O')

**In `CNT`, 'Perm(Russian Federation)' should be 'Russian Federation'; 'Macao-China', 'China-Shanghai', 'Hong Kong-China' -> 'China'; 'Florida (USA)', 'Connecticut (USA)', 'Massachusetts (USA)' -> 'United States of America'; Chinese Taipei' -> Taiwan.**

**Define:** Use `replace`

**Code:**

In [14]:
all_cols_df.CNT = all_cols_df.CNT.replace('Perm(Russian Federation)', 'Russian Federation')
all_cols_df.CNT = all_cols_df.CNT.replace('Macao-China', 'China')
all_cols_df.CNT = all_cols_df.CNT.replace('China-Shanghai', 'China')
all_cols_df.CNT = all_cols_df.CNT.replace('Hong Kong-China', 'China')
all_cols_df.CNT = all_cols_df.CNT.replace('Florida (USA)', 'United States of America')
all_cols_df.CNT = all_cols_df.CNT.replace('Connecticut (USA)', 'United States of America')
all_cols_df.CNT = all_cols_df.CNT.replace('Massachusetts (USA)', 'United States of America')
all_cols_df.CNT = all_cols_df.CNT.replace('Chinese Taipei', 'Taiwan')

**Check:**

In [15]:
all_cols_df.CNT.unique()

array(['Albania', 'United Arab Emirates', 'Argentina', 'Australia',
       'Austria', 'Belgium', 'Bulgaria', 'Brazil', 'Canada',
       'Switzerland', 'Chile', 'Colombia', 'Costa Rica', 'Czech Republic',
       'Germany', 'Denmark', 'Spain', 'Estonia', 'Finland', 'France',
       'United Kingdom', 'Greece', 'China', 'Croatia', 'Hungary',
       'Indonesia', 'Ireland', 'Iceland', 'Israel', 'Italy', 'Jordan',
       'Japan', 'Kazakhstan', 'Korea', 'Liechtenstein', 'Lithuania',
       'Luxembourg', 'Latvia', 'Mexico', 'Montenegro', 'Malaysia',
       'Netherlands', 'Norway', 'New Zealand', 'Peru', 'Poland',
       'Portugal', 'Qatar', 'Russian Federation',
       'United States of America', 'Romania', 'Singapore', 'Serbia',
       'Slovak Republic', 'Slovenia', 'Sweden', 'Taiwan', 'Thailand',
       'Tunisia', 'Turkey', 'Uruguay', 'Vietnam'], dtype=object)

**We could change the numerical values of `ST115Q01` by the definition of those numbers.**

**Define:** Use `replace`

**Code:**

In [16]:
# ST115Q01 replace
all_cols_df.ST115Q01 = all_cols_df.ST115Q01.replace(1, 'Never') # In order to avoid mistakes, I'll write "Never"
                                                                # instead of "None"
all_cols_df.ST115Q01 = all_cols_df.ST115Q01.replace(2, 'One or Two Times')
all_cols_df.ST115Q01 = all_cols_df.ST115Q01.replace(3, 'Three or Four Times')
all_cols_df.ST115Q01 = all_cols_df.ST115Q01.replace(4, 'Five or More Times')

/Users/home/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


**Check:**

In [17]:
all_cols_df.ST115Q01.unique()

array(['Never', 'One or Two Times', 'Three or Four Times', nan,
       'Five or More Times'], dtype=object)

**Values of `ST08Q01` and `ST09Q01` have more spaces than necessary. Example: 'None&nbsp;&nbsp;&nbsp;&nbsp;'.**

**Define:** `Use replace`

**Code:**

In [18]:
# ST08Q01
all_cols_df.ST08Q01 = all_cols_df.ST08Q01.replace('None  ', 'Never')
all_cols_df.ST08Q01 = all_cols_df.ST08Q01.replace('One or two times  ', 'One or Two Times')
all_cols_df.ST08Q01 = all_cols_df.ST08Q01.replace('Three or four times  ', "Three or Four Times")
all_cols_df.ST08Q01 = all_cols_df.ST08Q01.replace('Five or more times  ', 'Five or More Times')

# ST09Q01
all_cols_df.ST09Q01 = all_cols_df.ST09Q01.replace('None  ', 'Never')
all_cols_df.ST09Q01 = all_cols_df.ST09Q01.replace('One or two times  ', 'One or Two Times')
all_cols_df.ST09Q01 = all_cols_df.ST09Q01.replace('Three or four times  ', "Three or Four Times")
all_cols_df.ST09Q01 = all_cols_df.ST09Q01.replace('Five or more times  ', 'Five or More Times')

**Check:**

In [19]:
all_cols_df.ST08Q01.unique()

array(['Never', 'One or Two Times', 'Three or Four Times', nan,
       'Five or More Times'], dtype=object)

In [20]:
all_cols_df.ST08Q01.unique()

array(['Never', 'One or Two Times', 'Three or Four Times', nan,
       'Five or More Times'], dtype=object)

**`ST26Q06` 'Possessions - Internet' and `IC01Q04` 'At Home - Internet connection' seem to represent the same.**

In [21]:
print(pisa_dict['ST26Q06'])
pisa_df.ST26Q06.value_counts()

Possessions - Internet


Yes    402040
No      71142
Name: ST26Q06, dtype: int64

In [22]:
print(pisa_dict['IC01Q04'])
pisa_df.IC01Q04.value_counts()

At Home - Internet connection


Yes, and I use it          259762
No                          30161
Yes, but I don’t use it      7382
Name: IC01Q04, dtype: int64

`ST26Q06` has more values than` IC01Q04`, but `IC01Q04` besides asking if the student has internet or not, ask if they use it. Since I want to know if access to the Internet has any relationship with the student's performance, then it is necessary to know not only if a student has internet at home, but also to know if they use it. Therefore, we will remove `ST26Q06` and we will keep` IC01Q04`.

**Define:** Remove `ST26Q06` from df.

**Code:**

In [23]:
all_cols_df.drop('ST26Q06', axis=1, inplace = True)

/Users/home/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


**Check:**

In [24]:
'ST26Q06' in all_cols_df

False

**`ST04Q01` is a categorical variable.**

**Define:** Use `pd.api.types.CategoricalDtype`

**Code:**

In [25]:
# Gender
# Categories
genders = ["Female", "Male"]
gclasses = pd.api.types.CategoricalDtype(ordered = False, categories=genders)

# Convert
all_cols_df.ST04Q01 = all_cols_df.ST04Q01.astype(gclasses);

**Check:**

In [26]:
def print_dtypes_value_count_shape(col):
    print("____"*10, "\n" + pisa_dict[col])
    print(all_cols_df[col].dtypes)
    print(all_cols_df[col].shape)
    print(all_cols_df[col].value_counts())
    

print_dtypes_value_count_shape('ST04Q01')

________________________________________ 
Gender
category
(485490,)
Female    245064
Male      240426
Name: ST04Q01, dtype: int64


**`IMMIG`, `IC01Q04`, `ST08Q01`, `ST09Q01` are categorical ordinal variables, not strings.**

**Define:** Use `pd.api.types.CategoricalDtype`

**Code:**

In [27]:
# IMMIG
# Categories
immigration_status = ['Native', 'First-Generation', 'Second-Generation']
isclasses = pd.api.types.CategoricalDtype(ordered = True, categories=immigration_status)

# Convert
all_cols_df.IMMIG = all_cols_df.IMMIG.astype(isclasses);

In [28]:
# IC01Q04
# Categories
use_of_tech = ['Yes, and I use it', 'Yes, but I don’t use it', 'No']
use_of_tech_classes = pd.api.types.CategoricalDtype(ordered = True, categories=use_of_tech)

# Convert
all_cols_df.IC01Q04 = all_cols_df.IC01Q04.astype(use_of_tech_classes);

In [29]:
# ST08Q01
late_for_school = ['Never', 'One or Two Times', 'Three or Four Times', 'Five or More Times']
lclasses = pd.api.types.CategoricalDtype(ordered = True, categories=late_for_school)

# Convert
all_cols_df.ST08Q01 = all_cols_df.ST08Q01.astype(lclasses)
all_cols_df.ST09Q01 = all_cols_df.ST09Q01.astype(lclasses)
all_cols_df.ST115Q01 = all_cols_df.ST115Q01.astype(lclasses)

**Check:**

In [30]:
cat_ord_cols = ['IMMIG', 'IC01Q04', 'ST08Q01', 'ST09Q01', 'ST115Q01']

for col in cat_ord_cols:
    print_dtypes_value_count_shape(col)


________________________________________ 
Immigration status
category
(485490,)
Native               417347
Second-Generation     28268
First-Generation      26178
Name: IMMIG, dtype: int64
________________________________________ 
At Home - Internet connection
category
(485490,)
Yes, and I use it          259762
No                          30161
Yes, but I don’t use it      7382
Name: IC01Q04, dtype: int64
________________________________________ 
Truancy - Late for School
category
(485490,)
Never                  306065
One or Two Times       124380
Three or Four Times     29817
Five or More Times      18881
Name: ST08Q01, dtype: int64
________________________________________ 
Truancy - Skip whole school day
category
(485490,)
Never                  385998
One or Two Times        75969
Three or Four Times     10882
Five or More Times       6282
Name: ST09Q01, dtype: int64
________________________________________ 
Truancy - Skip classes within school day
category
(485490,)
Never      

**Columns have difficult names to remember**

**Define:** Use `rename`

**Code:**

In [31]:
new_names = {'STIDSTD': 'student_id', 'CNT' : 'country', 'ST04Q01': 'gender', 
             'IMMIG': 'immig_status', 'IC01Q04': 'internet_home','ST08Q01': 'late_to_school', 
             'ST115Q01': 'skip_class_within_school', 'ST09Q01': 'skip_whole_school_day', 
             'PV1MATH': 'math_score', 'PV1READ': 'reading_score', 'PV1SCIE': 'science_score'}

In [32]:
all_cols_df.rename(columns = new_names, inplace=True)

/Users/home/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


**It's possible that to make an analysis taking geographic variables, we'll need to have a column of a continent or a global region, not just a country**

**Define:** Create a dictionary linking every country to a global region

**Code:**

In [33]:
continent = {'Spain': 'Europe', 'Croatia': 'Europe', 'Hungary': 'Europe', 'Czech Republic': 'Europe',
             'United Kingdom': 'Europe', 'Greece': 'Europe', 'Belgium': 'Europe', 'Bulgaria': 'Europe', 
             'Germany': 'Europe', 'Denmark': 'Europe', 'Estonia': 'Europe', 'Finland': 'Europe', 'France': 'Europe',
             'Luxembourg': 'Europe', 'Latvia': 'Europe', 'Netherlands': 'Europe', 'Norway': 'Europe', 
             'Romania': 'Europe', 'Sweden': 'Europe', 'Russian Federation': 'Europe', 'Portugal': 'Europe',
             'Poland': 'Europe', 'Liechtenstein': 'Europe', 'Lithuania': 'Europe', 'Ireland': 'Europe', 
             'Iceland': 'Europe', 'Italy': 'Europe', 'Slovenia': 'Europe', 'Austria': 'Europe', 'Albania': 'Europe',
             'Montenegro': 'Europe', 'Slovak Republic': 'Europe', 'Serbia': 'Europe', 'Switzerland': 'Europe',
             'Argentina': 'Latin America', 'Brazil': 'Latin America', 'Chile': 'Latin America', 
             'Colombia': 'Latin America', 'Costa Rica': 'Latin America', 'Mexico': 'Latin America', 
             'Peru': 'Latin America', 'Uruguay': 'Latin America', 'Tunisia': 'Middle East and Africa',
             'United States of America': 'North America', 'Canada': 'North America',
             'Australia': 'Oceania', 'New Zealand': 'Oceania', 'Israel': 'Middle East and Africa', 'Qatar': 
             'Middle East and Africa', 'Vietnam': 'Middle East and Africa', 'Jordan': 'Middle East and Africa', 
             'Turkey': 'Middle East and Africa', 
             'United Arab Emirates': 'Middle East and Africa', 'Taiwan': 'Asia', 'China': 'Asia', 'Japan': 'Asia', 
             'Korea': 'Asia', 'Singapore': 'Asia', 'Thailand': 'Asia', 'Malaysia': 'Asia', 'Indonesia': 'Asia', 
             'Kazakhstan': 'Asia'}

In [34]:
# Create global_region col
all_cols_df['global_region'] = all_cols_df.apply(lambda row: continent[row.country], axis=1)

/Users/home/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [35]:
# Convert global_regions to a categorical type
global_regions = ['Europe', 'Latin America', 'North America', 'Asia', 'Middle East and Africa', 'Oceania']
region_classes = pd.api.types.CategoricalDtype(ordered = False, categories=global_regions)

# Convert
all_cols_df.global_region = all_cols_df.global_region.astype(region_classes)

**Check:**

In [36]:
all_cols_df['global_region'].value_counts()

Europe                    233917
Latin America              90799
Asia                       61391
Middle East and Africa     48773
North America              31838
Oceania                    18772
Name: global_region, dtype: int64

In [37]:
all_cols_df.global_region.dtypes

CategoricalDtype(categories=['Europe', 'Latin America', 'North America', 'Asia',
                  'Middle East and Africa', 'Oceania'],
                 ordered=False)

**Due to the amount of data we have in this dataset, we consider it prudent to have a column called `comb_score`, which is the sum of the three scores obtained by the student.**

**Define:** I'll create a new columns using `apply` and `np.sum`.

**Code:**

In [38]:
all_cols_df['comb_score'] = all_cols_df.apply(lambda row: 
                                         np.sum([row.math_score, row.reading_score, row.science_score]), axis=1)

/Users/home/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


**Check:**

In [39]:
all_cols_df.sample(3)

,student_id,gender,country,ESCS,science_score,reading_score,math_score,immig_status,late_to_school,skip_whole_school_day,skip_class_within_school,internet_home,global_region,comb_score
84353,9884,Female,Canada,0.59,525.1212,525.2018,499.1510,Native,Never,One or Two Times,Never,NaN,North America,1549.4740
185124,621,Female,France,-0.68,562.0477,560.0720,535.0600,Native,One or Two Times,Never,Never,NaN,Europe,1657.1797
144558,6457,Female,Denmark,-0.14,435.5091,487.8692,466.8251,Native,Never,Never,Never,"Yes, and I use it",Europe,1390.2034


### Check everything again

In [40]:
all_cols_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485490 entries, 0 to 485489
Data columns (total 14 columns):
student_id                  485490 non-null object
gender                      485490 non-null category
country                     485490 non-null object
ESCS                        473648 non-null float64
science_score               485490 non-null float64
reading_score               485490 non-null float64
math_score                  485490 non-null float64
immig_status                471793 non-null category
late_to_school              479143 non-null category
skip_whole_school_day       479131 non-null category
skip_class_within_school    479269 non-null category
internet_home               297305 non-null category
global_region               485490 non-null category
comb_score                  485490 non-null float64
dtypes: category(7), float64(5), object(2)
memory usage: 29.2+ MB


## Exporting Data

In [41]:
all_cols_df.to_csv('clean_pisa_data.csv', index=False)

# Export column types too
clean_pisa_series = all_cols_df.dtypes
clean_pisa_series.to_pickle('clean_pisa_cols.pkl')